Homework 1:

Using chinook.db write pandas code.

Customer Purchases Analysis:


In [1]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np

# Define connection string
server = 'DESKTOP-4EHVA5N\SQLEXPRESS'
database = 'Chinook'
driver = 'ODBC Driver 17 for SQL Server'

# Build connection URL for SQLAlchemy
connection_url = f"mssql+pyodbc://@{server}/{database}?trusted_connection=yes&driver={driver.replace(' ', '+')}"

# Create SQLAlchemy engine
engine = create_engine(connection_url)

# Read data from a table
df = pd.read_sql("SELECT * FROM Customer", engine)
df1 = pd.read_sql("SELECT * FROM Invoice", engine)

# Display the data
print(df.head())


   CustomerId  FirstName     LastName  \
0           1       Luís    Gonçalves   
1           2     Leonie       Köhler   
2           3   François     Tremblay   
3           4      Bjørn       Hansen   
4           5  František  Wichterlová   

                                            Company  \
0  Embraer - Empresa Brasileira de Aeronáutica S.A.   
1                                              None   
2                                              None   
3                                              None   
4                                  JetBrains s.r.o.   

                           Address                 City State         Country  \
0  Av. Brigadeiro Faria Lima, 2170  São José dos Campos    SP          Brazil   
1          Theodor-Heuss-Straße 34            Stuttgart  None         Germany   
2                1498 rue Bélanger             Montréal    QC          Canada   
3                 Ullevålsveien 14                 Oslo  None          Norway   
4                

In [2]:
# Find the total amount spent by each customer on purchases (considering invoices).

In [3]:
df.to_csv('Customer.csv',index=False)

In [4]:
df1.to_csv('Invoice.csv',index=False)

In [5]:
query = '''SELECT 
                CustomerId, SUM(Total) as TotalSpent
                FROM Invoice
                Group by CustomerId
                ORDER BY TotalSpent DESC;'''

In [6]:
df = pd.read_sql(query, engine)
df

,CustomerId,TotalSpent
0,6,49.62
1,26,47.62
2,57,46.62
3,45,45.62
4,46,45.62
5,37,43.62
6,28,43.62
7,24,43.62
8,25,42.62
9,7,42.62


In [7]:
# Identify the top 5 customers with the highest total purchase amounts.
query1 = '''SELECT 
    c.CustomerId,
    c.FirstName + ' ' + c.LastName AS CustomerName,
    SUM(i.Total) AS TotalSpent
FROM 
    Invoice i
JOIN 
    Customer c ON i.CustomerId = c.CustomerId
GROUP BY 
    c.CustomerId, c.FirstName, c.LastName
ORDER BY 
    TotalSpent DESC
OFFSET 0 ROWS FETCH NEXT 5 ROWS ONLY'''

In [8]:
df = pd.read_sql(query1, engine)
df

,CustomerId,CustomerName,TotalSpent
0,6,Helena Holý,49.62
1,26,Richard Cunningham,47.62
2,57,Luis Rojas,46.62
3,45,Ladislav Kovács,45.62
4,46,Hugh O'Reilly,45.62


In [9]:
# Display the customer ID, name, and the total amount spent for the top 5 customers.

query2 = '''SELECT 
    c.CustomerId,
    c.FirstName,
    SUM(i.Total) AS TotalSpent
FROM 
    Invoice i
JOIN 
    Customer c ON i.CustomerId = c.CustomerId
GROUP BY 
    c.CustomerId, c.FirstName
ORDER BY 
    TotalSpent DESC
OFFSET 0 ROWS FETCH NEXT 5 ROWS ONLY'''

In [10]:
df = pd.read_sql(query2, engine)
df

,CustomerId,FirstName,TotalSpent
0,6,Helena,49.62
1,26,Richard,47.62
2,57,Luis,46.62
3,45,Ladislav,45.62
4,46,Hugh,45.62


Album vs. Individual Track Purchases:

In [11]:
# Determine the percentage of customers who prefer to buy individual tracks instead of full albums.

query3 = '''WITH AllCustomers AS (
    SELECT CustomerId FROM Customer
),
CustomerTrackPurchases AS (
    SELECT i.CustomerId, t.AlbumId, il.TrackId
    FROM Invoice i
    JOIN InvoiceLine il ON i.InvoiceId = il.InvoiceId
    JOIN Track t ON il.TrackId = t.TrackId
),
AlbumTracks AS (
    SELECT AlbumId, COUNT(*) AS TotalTracks
    FROM Track
    GROUP BY AlbumId
),
CustomerAlbumPurchases AS (
    SELECT CustomerId, AlbumId, COUNT(DISTINCT TrackId) AS TracksBought
    FROM CustomerTrackPurchases
    GROUP BY CustomerId, AlbumId
),
FullAlbumBuyers AS (
    SELECT cap.CustomerId
    FROM CustomerAlbumPurchases cap
    JOIN AlbumTracks at ON cap.AlbumId = at.AlbumId
    WHERE cap.TracksBought = at.TotalTracks
),
TrackBuyers AS (
    SELECT ac.CustomerId
    FROM AllCustomers ac
    LEFT JOIN FullAlbumBuyers fab ON ac.CustomerId = fab.CustomerId
    WHERE fab.CustomerId IS NULL
)
SELECT 
    (SELECT COUNT(*) FROM AllCustomers) AS TotalCustomers,
    (SELECT COUNT(DISTINCT CustomerId) FROM FullAlbumBuyers) AS AlbumBuyers,
    (SELECT COUNT(*) FROM TrackBuyers) AS TrackBuyers,
    CAST((SELECT COUNT(*) FROM TrackBuyers) AS FLOAT) * 100.0 / 
    (SELECT COUNT(*) FROM AllCustomers) AS TrackBuyerPercentage;'''

In [12]:
df = pd.read_sql(query3, engine)
df

,TotalCustomers,AlbumBuyers,TrackBuyers,TrackBuyerPercentage
0,59,21,38,64.40678


In [13]:
# A customer is considered to prefer individual tracks if they have purchased only a subset of tracks from an album.
query4 = """ -- Total customers
WITH AllCustomers AS (
    SELECT CustomerId FROM Customer
),

-- Each track purchased by customer
CustomerTracks AS (
    SELECT i.CustomerId, t.AlbumId, il.TrackId
    FROM Invoice i
    JOIN InvoiceLine il ON i.InvoiceId = il.InvoiceId
    JOIN Track t ON il.TrackId = t.TrackId
),

-- Total number of tracks in each album
AlbumTracks AS (
    SELECT AlbumId, COUNT(*) AS TotalTracks
    FROM Track
    GROUP BY AlbumId
),

-- Count of tracks bought per album by customer
CustomerAlbumTracks AS (
    SELECT CustomerId, AlbumId, COUNT(DISTINCT TrackId) AS TracksBought
    FROM CustomerTracks
    GROUP BY CustomerId, AlbumId
),

-- Customers who bought *all* tracks in at least one album
FullAlbumBuyers AS (
    SELECT cat.CustomerId
    FROM CustomerAlbumTracks cat
    JOIN AlbumTracks at ON cat.AlbumId = at.AlbumId
    WHERE cat.TracksBought = at.TotalTracks
),

-- Customers who never bought a full album
TrackOnlyBuyers AS (
    SELECT ac.CustomerId
    FROM AllCustomers ac
    LEFT JOIN FullAlbumBuyers fab ON ac.CustomerId = fab.CustomerId
    WHERE fab.CustomerId IS NULL
)

-- Final result: percentage of track-only buyers
SELECT 
    (SELECT COUNT(*) FROM TrackOnlyBuyers) * 100.0 / COUNT(*) AS TrackBuyerPercentage
FROM AllCustomers;
 """


In [14]:
df = pd.read_sql(query4, engine)
df

,TrackBuyerPercentage
0,64.40678


In [15]:
# Provide a summary of the percentage of customers who fall into each category (individual tracks vs. full albums).
query5 = """ -- Step 1: All customers
WITH AllCustomers AS (
    SELECT CustomerId FROM Customer
),

-- Step 2: What each customer bought
CustomerTracks AS (
    SELECT i.CustomerId, t.AlbumId, il.TrackId
    FROM Invoice i
    JOIN InvoiceLine il ON i.InvoiceId = il.InvoiceId
    JOIN Track t ON il.TrackId = t.TrackId
),

-- Step 3: How many tracks are in each album
AlbumTracks AS (
    SELECT AlbumId, COUNT(*) AS TotalTracks
    FROM Track
    GROUP BY AlbumId
),

-- Step 4: How many tracks each customer bought per album
CustomerAlbumTracks AS (
    SELECT CustomerId, AlbumId, COUNT(DISTINCT TrackId) AS TracksBought
    FROM CustomerTracks
    GROUP BY CustomerId, AlbumId
),

-- Step 5: Who bought all tracks of any album (full album buyers)
FullAlbumBuyers AS (
    SELECT DISTINCT cat.CustomerId
    FROM CustomerAlbumTracks cat
    JOIN AlbumTracks at ON cat.AlbumId = at.AlbumId
    WHERE cat.TracksBought = at.TotalTracks
),

-- Step 6: Track-only buyers (no full albums)
TrackOnlyBuyers AS (
    SELECT ac.CustomerId
    FROM AllCustomers ac
    LEFT JOIN FullAlbumBuyers fab ON ac.CustomerId = fab.CustomerId
    WHERE fab.CustomerId IS NULL
)

-- Final output: total, full album, and track-only counts + percentages
SELECT
    (SELECT COUNT(*) FROM FullAlbumBuyers) AS FullAlbumBuyers,
    (SELECT COUNT(*) FROM TrackOnlyBuyers) AS TrackOnlyBuyers,
    COUNT(*) AS TotalCustomers,
    ROUND((SELECT COUNT(*) FROM FullAlbumBuyers) * 100.0 / COUNT(*), 2) AS FullAlbumPercentage,
    ROUND((SELECT COUNT(*) FROM TrackOnlyBuyers) * 100.0 / COUNT(*), 2) AS TrackOnlyPercentage
FROM AllCustomers;
 """

In [16]:
df = pd.read_sql(query5, engine)
df

,FullAlbumBuyers,TrackOnlyBuyers,TotalCustomers,FullAlbumPercentage,TrackOnlyPercentage
0,21,38,59,35.59,64.41
